<a href="https://colab.research.google.com/github/ghr8635/E2E-DriveAI-ROS2-based-Modular-Framework-for-Autonomous-Vehicle-Control/blob/main/inference_from_resent_feature_extractor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
import torch
import torch.nn as nn
from torchvision import models, transforms
from PIL import Image

In [2]:
my_model = '/home/linux1/E2E-DriveAI-ROS2-based-Modular-Framework-for-Autonomous-Vehicle-Control/resnet18/trained models/feature_extractor_resnet18.pth'

model = models.resnet18(pretrained=False)
model.fc = nn.Linear(model.fc.in_features, 10)
model.load_state_dict(torch.load(my_model))
model.fc = torch.nn.Identity()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

/home/linux1/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/linux1/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
/tmp/ipykernel_158207/887775720.py:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This

In [4]:
my_image = '/home/linux1/my_image.png'

transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=3),  # Convert 1-channel to 3-channel
    transforms.Resize((224, 224)),  # Resize images to 224x224
    transforms.ToTensor(),  # Convert to Tensor
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # Normalize the image
])
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

img = Image.open(my_image)

# Apply transformations
img = transform(img)

# Add batch dimension
img = img.unsqueeze(0)
img = img.to(device)




In [7]:
features = model(img)

H = 62          #these values are obtaiend from pointpillar inference, in order to match 
W = 54
resnet_features_repeated = features.unsqueeze(1).repeat(1, H * W, 1)  # [1, 3348, 512]

print(resnet_features_repeated.shape)
print(resnet_features_repeated)

torch.Size([1, 3348, 512])
tensor([[[0.8062, 0.7448, 0.6932,  ..., 0.8805, 0.9399, 0.8934],
         [0.8062, 0.7448, 0.6932,  ..., 0.8805, 0.9399, 0.8934],
         [0.8062, 0.7448, 0.6932,  ..., 0.8805, 0.9399, 0.8934],
         ...,
         [0.8062, 0.7448, 0.6932,  ..., 0.8805, 0.9399, 0.8934],
         [0.8062, 0.7448, 0.6932,  ..., 0.8805, 0.9399, 0.8934],
         [0.8062, 0.7448, 0.6932,  ..., 0.8805, 0.9399, 0.8934]]],
       grad_fn=<RepeatBackward0>)
